In [2]:
import pandas as pd
import numpy as np
from functools import partial
from ctcv_helper import print_table
from ctcv_helper import read_dataframe
from ctcv_helper import read_dataframe_ratio

In [3]:
# Read limits from csv files
limits = {}
for model in ['K6']:
    df = pd.DataFrame.from_csv("Mar28_unblinded_rangefix/limits_cv_0p5.csv", sep=",", index_col=None)
    df = df.append(pd.DataFrame.from_csv("Mar28_unblinded_rangefix/limits_cv_1p0.csv", sep=",", index_col=None), ignore_index=True)
    df = df.append(pd.DataFrame.from_csv("Mar28_unblinded_rangefix/limits_cv_1p5.csv", sep=",", index_col=None), ignore_index=True)
    limits[model] = df

In [4]:
# Read the cross sections from csv files
df_xsbr = {}
df_br = {}
df_xsecs = {}
for model in ['K4', 'K5', 'K6']:
    df_xsbr[model]  = pd.DataFrame.from_csv("xsbr_scalings_%s.csv"%model, sep=",", index_col=None)
    df_br[model]    = pd.DataFrame.from_csv("br_scalings_%s.csv"%model,   sep=",", index_col=None)
    df_xsecs[model] = pd.DataFrame.from_csv("xsecs_tH_ttH_%s.csv"%model, sep=",", index_col=None)

In [5]:
# Print pure limits on r
MOD = 'K6'
print " alpha  Ct/CV    CV=0.5     CV=1.0     CV=1.5"
print "                  exp   (obs)     exp   (obs)     exp   (obs)"
print_table([partial(read_dataframe, df=limits[MOD], att='exp'),
             partial(read_dataframe, df=limits[MOD], att='obs')],
             linepat=" %6.3f (%6.3f)")

 alpha  Ct/CV    CV=0.5     CV=1.0     CV=1.5
                  exp   (obs)     exp   (obs)     exp   (obs)
-0.973 -6.000:   0.015 ( 0.026)                                
-0.941 -4.000:   0.038 ( 0.070)                                
-0.900 -3.000:   0.076 ( 0.144)  0.076 ( 0.144)                
-0.862 -2.500:   0.117 ( 0.227)                                
-0.800 -2.000:   0.194 ( 0.385)  0.194 ( 0.385)  0.194 ( 0.385)
-0.692 -1.500:   0.352 ( 0.715)  0.352 ( 0.715)                
-0.640 -1.333:                                   0.443 ( 0.899)
-0.610 -1.250:                   0.494 ( 1.004)                
-0.500 -1.000:   0.709 ( 1.437)  0.709 ( 1.437)  0.709 ( 1.437)
-0.410 -0.833:                                   0.926 ( 1.822)
-0.360 -0.750:                   1.051 ( 2.027)                
-0.308 -0.667:                                   1.191 ( 2.239)
-0.200 -0.500:   1.570 ( 2.674)  1.570 ( 2.673)  1.570 ( 2.673)
-0.100 -0.333:                                   2.055 ( 3.0

In [6]:
# Print limit * cross section * BR
def xs_limit(ct,cv, df_xsecs, df_limits, limval='exp'):
    # Take the cross section for cv=1.0, ct=ct/cv
    xsec = read_dataframe(round(ct/cv,3), 1.0, df=df_xsecs, att='tot')
    lim  = read_dataframe(ct, cv, df=df_limits, att=limval)
    return float(xsec)*float(lim)

print " alpha  Ct/CV    CV=0.5     CV=1.0     CV=1.5"
print "                  exp   (obs)     exp   (obs)     exp   (obs)"
print_table([partial(xs_limit, df_xsecs=df_xsecs[MOD], df_limits=limits[MOD], limval='exp'),
             partial(xs_limit, df_xsecs=df_xsecs[MOD], df_limits=limits[MOD], limval='obs')],
             linepat=" %6.3f (%6.3f) ")

 alpha  Ct/CV    CV=0.5     CV=1.0     CV=1.5
                  exp   (obs)     exp   (obs)     exp   (obs)
-0.973 -6.000:   0.335 ( 0.606)                                   
-0.941 -4.000:   0.337 ( 0.626)                                   
-0.900 -3.000:   0.335 ( 0.639)   0.335 ( 0.639)                  
-0.862 -2.500:   0.334 ( 0.649)                                   
-0.800 -2.000:   0.330 ( 0.656)   0.330 ( 0.656)   0.330 ( 0.656) 
-0.692 -1.500:   0.325 ( 0.660)   0.325 ( 0.660)                  
-0.640 -1.333:                                     0.325 ( 0.659) 
-0.610 -1.250:                    0.321 ( 0.653)                  
-0.500 -1.000:   0.315 ( 0.638)   0.315 ( 0.638)   0.315 ( 0.638) 
-0.410 -0.833:                                     0.312 ( 0.615) 
-0.360 -0.750:                    0.307 ( 0.593)                  
-0.308 -0.667:                                     0.301 ( 0.566) 
-0.200 -0.500:   0.292 ( 0.497)   0.292 ( 0.497)   0.292 ( 0.497) 
-0.100 -0.333:       

#### Now do this again, but store it in a dataframe and write it to a csv file

In [8]:
df_xs_limits = pd.DataFrame()
df_xs_limits['alpha'] = np.sign(limits[MOD].cf*limits[MOD].cv)*limits[MOD].cf**2/(limits[MOD].cf**2+limits[MOD].cv**2)
df_xs_limits['alpha'] = df_xs_limits.alpha.round(4)
df_xs_limits['ratio'] = limits[MOD].cf/limits[MOD].cv
df_xs_limits['ratio'] = df_xs_limits.ratio.round(3)

# Use our xs_limit function to store the limits
for limval in ['twosigdown', 'onesigdown', 'exp', 'onesigup', 'twosigup', 'obs']:
    df_xs_limits[limval] = np.vectorize(partial(xs_limit,
                                               df_xsecs=df_xsecs[MOD],
                                               df_limits=limits[MOD],
                                               limval=limval))(limits[MOD].cf,
                                                                    limits[MOD].cv)

# Remove the duplicate entries, sort, reset the index, and write to csv
df_xs_limits.drop_duplicates(subset='ratio', inplace=True)
df_xs_limits.sort_values(by='ratio', inplace=True)
df_xs_limits.index = range(1,len(df_xs_limits)+1)
df_xs_limits.to_csv("xs_limits_%s.csv"%MOD)

df_xs_limits.loc[df_xs_limits.ratio==1.0]

,alpha,ratio,twosigdown,onesigdown,exp,onesigup,twosigup,obs
25,0.5,1,0.128112,0.171949,0.24383,0.348791,0.485304,0.561915


#### Produce also a csv file with the pure r limits (for cv == 1)

In [22]:
df_r_limits = pd.DataFrame()
df_r_limits['alpha'] = np.sign(limits[MOD].cf*limits[MOD].cv)*limits[MOD].cf**2/(limits[MOD].cf**2+limits[MOD].cv**2)
df_r_limits['alpha'] = df_r_limits.alpha.round(4)
df_r_limits['ratio'] = limits[MOD].cf/limits[MOD].cv
df_r_limits['ratio'] = df_r_limits.ratio.round(3)

# Use our xs_limit function to store the limits
for limval in ['twosigdown', 'onesigdown', 'exp', 'onesigup', 'twosigup', 'obs']:
    df_r_limits[limval] = getattr(limits[MOD], limval)

# Remove the duplicate entries, sort, reset the index, and write to csv
df_r_limits.drop_duplicates(subset='ratio', inplace=True)
df_r_limits.sort_values(by='ratio', inplace=True)
df_r_limits.index = range(1,len(df_xs_limits)+1)
df_r_limits.to_csv("r_limits_%s.csv"%MOD)


In [23]:
df_r_limits

,alpha,ratio,twosigdown,onesigdown,exp,onesigup,twosigup,obs
1,-0.9730,-6.000,0.0075,0.0104,0.0142,0.0200,0.0276,0.0261
2,-0.9412,-4.000,0.0203,0.0270,0.0376,0.0535,0.0730,0.0704
3,-0.9000,-3.000,0.0405,0.0540,0.0757,0.1068,0.1468,0.1443
4,-0.8621,-2.500,0.0620,0.0829,0.1167,0.1655,0.2269,0.2269
5,-0.8000,-2.000,0.1026,0.1379,0.1938,0.2765,0.3780,0.3846
6,-0.6923,-1.500,0.1852,0.2493,0.3525,0.5029,0.6963,0.7149
7,-0.6400,-1.333,0.2312,0.3116,0.4434,0.6325,0.8757,0.8986
8,-0.6098,-1.250,0.2577,0.3482,0.4941,0.7088,0.9786,1.0045
9,-0.5000,-1.000,0.3670,0.4992,0.7090,1.0283,1.4292,1.4367
10,-0.4098,-0.833,0.4719,0.6439,0.9258,1.3354,1.8730,1.8220
